In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras import backend as K
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Template for Keras tasks of sheet 5

In [ ]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
img_rows, img_cols = 28, 28

# Normalize it
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
# Store it in the correct format for Keras
# The image data has a single channel (grayscale values)
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
# Store the labels in the correct format for Keras
Y_train = keras.utils.np_utils.to_categorical(y_train, 10)
Y_test = keras.utils.np_utils.to_categorical(y_test, 10)

To use `X_train` for fully-connected inputs, reshape it. Use the `input_shape` variable in the first layer of your networks.

In [9]:
def plot_history(history):
    """Create a plot showing the training history of `model.fit`.
    
    Example:
        history = model.fit(...)
        plot_history(history)
    """
    x = range(history.params['epochs'])
    acc, val_acc = history.history['acc'], history.history.get('val_acc')
    f, axarr = plt.subplots(2, sharex=True)
    axarr[0].set_title('accuracy')
    axarr[0].plot(x, acc, label='train')
    if val_acc:
        axarr[0].plot(x, val_acc, label='validation')
    axarr[0].legend()
    
    loss, val_loss = history.history['loss'], history.history.get('val_loss')
    axarr[1].set_title('loss')
    axarr[1].plot(x, loss, label='train')
    if val_loss:
        axarr[1].plot(x, val_loss, label='validation')
    axarr[1].legend()

In [ ]:
model1 = keras.models.Sequential()
model1.add(Dense(units=128, activation='relu', input_dim=28*28))
model1.add(Dense(units=128, activation='relu', input_dim=128))
model1.add(Dense(units=10, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])


history = model2.fit(X_train.reshape(60000,-1), Y_train.reshape(60000,-1), epochs=50, batch_size=128)
loss_and_metrics = model1.evaluate(X_test.reshape(10000,-1), Y_test.reshape(10000,-1), batch_size=128)

plot_history(history)
print(loss_and_metrics)

In [ ]:
model2 = keras.models.Sequential()
model2.add(Dense(units=128, activation='relu', input_dim=28*28))
model2.add(Dropout(0.3))
model2.add(Dense(units=128, activation='relu', input_dim=128))
model2.add(Dropout(0.3))
model2.add(Dense(units=10, activation='softmax'))
model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model2.fit(X_train.reshape(60000,-1), Y_train.reshape(60000,-1), epochs=250, batch_size=128)
loss_and_metrics = model2.evaluate(X_test.reshape(10000,-1), Y_test.reshape(10000,-1), batch_size=128)

plot_history(history)
print(loss_and_metrics)

In [ ]:
model3 = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model3.add(Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model3.add(Conv2D(32, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(10, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model3 .fit(X_train, Y_train, batch_size=128, epochs=15)
score = model3.evaluate(X_test, Y_test, batch_size=32)
plot_history(history)
print(score)

## Pedestrian Bonus Task

In [2]:
from sklearn.svm import LinearSVC
import scipy.io
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [3]:
ped_data = scipy.io.loadmat("data/pca_ped_25x50.mat")
ped_images = ped_data['ped_train_int_25x50'][:,1:]
garb_images = ped_data['garb_train_int_25x50'][:,1:]
ped_test_images = ped_data['ped_test_int_25x50'][:,1:]
garb_test_images = ped_data['garb_test_int_25x50'][:,1:]
del ped_data


min_max = MinMaxScaler()
min_max.fit(np.vstack((ped_images, garb_images)))

ped_images = min_max.transform(ped_images)
garb_images = min_max.transform(garb_images)
y_train = np.concatenate((np.ones(len(ped_images)), np.zeros(len(garb_images))))

ped_test_images = min_max.transform(ped_test_images)
garb_test_images = min_max.transform(garb_test_images)
y_test = np.concatenate((np.ones(len(ped_test_images)), np.zeros(len(garb_test_images))))

In [4]:
img_rows, img_cols = 25, 50
X_train = np.vstack((ped_images, garb_images)).astype('float32')
X_test = np.vstack((ped_test_images, garb_test_images)).astype('float32')
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
Y_train = keras.utils.np_utils.to_categorical(y_train,2)
Y_test = keras.utils.np_utils.to_categorical(y_test, 2)

print(Y_train.shape)

(3000, 2)


In [6]:
import densenet

# 'th' dim-ordering or 'tf' dim-ordering
image_dim = (25, 50, 1)

model = densenet.DenseNet(classes=2, input_shape=image_dim, depth=40, growth_rate=12, 
            bottleneck=True, reduction=0.5)




In [7]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print("starting training")
model.fit(X_train, Y_train, batch_size=32, epochs=10)
score = model.evaluate(X_test, Y_test, batch_size=32)
print(score)

starting training
Epoch 1/10
3000/3000 [==============================] - 425s 142ms/step - loss: 0.6758 - acc: 0.7740
Epoch 2/10
3000/3000 [==============================] - 428s 143ms/step - loss: 0.4729 - acc: 0.8863
Epoch 3/10
3000/3000 [==============================] - 512s 171ms/step - loss: 0.3968 - acc: 0.9240
Epoch 4/10
3000/3000 [==============================] - 545s 182ms/step - loss: 0.3662 - acc: 0.9340
Epoch 5/10
3000/3000 [==============================] - 531s 177ms/step - loss: 0.3100 - acc: 0.9583
Epoch 6/10
3000/3000 [==============================] - 530s 177ms/step - loss: 0.2943 - acc: 0.9637
Epoch 7/10
3000/3000 [==============================] - 530s 177ms/step - loss: 0.2813 - acc: 0.9653
Epoch 8/10
3000/3000 [==============================] - 530s 177ms/step - loss: 0.2705 - acc: 0.9727
Epoch 9/10
3000/3000 [==============================] - 531s 177ms/step - loss: 0.2646 - acc: 0.9723
Epoch 10/10
1000/1000 [==============================] - 62s 62ms/step
[0

In [10]:
history = model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test, Y_test))
plot_history(history)

Train on 3000 samples, validate on 1000 samples
Epoch 1/10
3000/3000 [==============================] - 477s 159ms/step - loss: 0.2253 - acc: 0.9883 - val_loss: 1.2305 - val_acc: 0.8260
Epoch 2/10
3000/3000 [==============================] - 496s 165ms/step - loss: 0.2297 - acc: 0.9873 - val_loss: 1.0263 - val_acc: 0.7270
Epoch 3/10
1600/3000 [===============>..............] - ETA: 3:23 - loss: 0.2278 - acc: 0.9900

KeyboardInterrupt: 

In [11]:
model = densenet.DenseNetImageNet121(input_shape=image_dim)


ValueError: Negative dimension size caused by subtracting 2 from 1 for 'average_pooling2d_7/AvgPool' (op: 'AvgPool') with input shapes: [?,1,3,512].